## 소프트맥스 회귀의 비용 함수 구현하기(low-level)

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [4]:
z = torch.FloatTensor([1,2,3]) #3개의 원소를 가진 벡터 텐서

In [5]:
hypothesis = F.softmax(z, dim=0) #이 텐서를 소프트맥스 함수의 입력값으로 사용
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


결과값이 0과 1 사이의 벡터로 변환된 것을 확인할 수 있음. 합이 1인지도 확인

In [6]:
hypothesis.sum()

tensor(1.)

In [8]:
#비용함수 직접 구현
z = torch.rand(3, 5, requires_grad=True) #임의의 3x5 행렬 크기의 텐서

In [9]:
hypothesis = F.softmax(z,dim=1) #두 번째 차원에 함수를 적용한다는 의미로 dim=1
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward0>)


위의 텐서는 3개의 샘플에 대해서 5개의 클래스 중 어떤 클래스가 정답인지를 예측한 결과

In [12]:
y = torch.randint(5,(3,)).long() #임의 레이블 만듬
print(y)

tensor([3, 2, 3])


In [13]:
#각 레이블에 대해서 원-핫 인코딩 수행
y_one_hot = torch.zeros_like(hypothesis)  #모든 원소가 0인 3x5 텐서 생성
y_one_hot.scatter_(1, y.unsqueeze(1), 1) #unsqueeze(1)를 함으로서 3,크기였던 텐서는 3x1이 됨

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [14]:
print(y.unsqueeze(1))

tensor([[3],
        [2],
        [3]])


scatter의 인자로 첫번째 dim=1을 넣어주고 세번째 인자에 숫자 1을 넣어줌으로서 두번째 인자인 y.unsqueeze(1)이 알려주는 위치에 1을 넣는다  
'_' (언더바)는 덮어쓰기 연산

In [15]:
print(y_one_hot)

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])


In [17]:
#소프트맥스 회귀의 비용함수 구현
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6800, grad_fn=<MeanBackward0>)


## 소프트맥스 회귀의 비용 함수 구현하기(high-level)

### 1. F.softmax() + torch.log() = F.log_softmax()

앞선 코드에서 소프트맥스 함수의 결과에 로그를 씌울 때는 함수의 출력값을 로그 함수의 입력으로 사용하였다.

In [18]:
torch.log(F.softmax(z, dim=1))

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward0>)

파이토치에서는 이 두 함수를 결합한 **F.log_softmax()** 라는 도구를 제공한다.

In [19]:
F.log_softmax(z,dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward0>)

### 2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [20]:
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean() #low-level로 구현한 비용 함수

tensor(1.6800, grad_fn=<MeanBackward0>)

In [21]:
(y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean() #위에서 배운 함수로 대체

tensor(1.6800, grad_fn=<MeanBackward0>)

In [22]:
F.nll_loss(F.log_softmax(z, dim=1), y) #nll_loss를 사용할 때에는 원-핫 벡터 없이 바로 실제값을 인자로 사용한다

tensor(1.6800, grad_fn=<NllLossBackward0>)

nll은 Negative Log likelihood의 약자이다. 위에서 nll_loss는 F.log_softmax를 수행한 후에 남은 수식들을 수행한다. 

In [25]:
F.cross_entropy(z, y) #crooss_entropy는 log_softmax와 nll_loss를 포함한다

tensor(1.6800, grad_fn=<NllLossBackward0>)